---
title: Part 3. Actinia Process Chians
description: Learn how to create custom actinia process chain templates for GRASS GIS application.
format: html
author: 
    - Corey T. White
    - Vaclav Petras
date: '2024-09-10'
keep-ipynb: true
toc: true
toc-depth: 4
image:  images/webinar_title.webp
categories: [geospatial, GRASS v8.5, jupyter, lidar, STAC]
page-layout: full
title-block-banner: true
---

## Setup Environment

In [3]:
import os
import subprocess
from pprint import pprint
import sys
import json
import time

import requests
from requests.auth import HTTPBasicAuth

In [4]:
ACTINIA_USER = 'actinia-gdi'
ACTINIA_PASSWORD = 'actinia-gdi'
AUTH = 'actinia-gdi:actinia-gdi'
ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
# ACTINIA_BASEURL = 'https://openplains.app/actinia'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth(ACTINIA_USER, ACTINIA_PASSWORD)

In [5]:
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)
# create a temporary folder where to place our GRASS project
import tempfile
from pathlib import Path

tempdir = tempfile.TemporaryDirectory()

import grass.script as gs
import grass.jupyter as gj

gs.create_project(path=tempdir.name, name="foss4g2024_p2", epsg="2817", overwrite=True)
session = gj.init(Path(tempdir.name,"foss4g2024_p2"))

In [6]:
step_1_raw = !g.region raster=elevation res=10 --json
step_1 = ''.join(step_1_raw)
step_1_json = json.loads(step_1)
pprint(step_1_json)

{'id': 'g.region_1804289383',
 'inputs': [{'param': 'raster', 'value': 'elevation'},
            {'param': 'res', 'value': '10'},
            {'param': 'format', 'value': 'plain'}],
 'module': 'g.region'}


In [ ]:
g.region align=direction_3k_10m_d@basin_test n='252702.4094174725' e='618028.8126522538' s='229056.70015573464' w='598579.6858165322'
r.circle -b output=circle coordinate='601225.5895524962,241014.7657618352' max=200
r.stream.basins -c direction=direction_3k_10m_d@basin_test stream_rast=circle basins=point_basin_cloud memory=1500
r.to.vect input=point_basin_cloud output=point_basin_cloud type=area column=value
r.mask raster=point_basin_cloud
r.stats input='nlcd_2019@https://storage.googleapis.com/tomorrownow-actinia-dev/nlcd/nlcd_2019_cog.tif' separator="," output="point_basin_stats.csv" -alnpc
v.out.ogr in=point_basin_cloud output='PG:host=db port=5432 dbname=actinia user=actinia password=actinia' format=PostgreSQL type=area


## Output formats

- **kv**: parses the module output and creates key/value pairs
- **list**: parses the module output and creates a list of values
- **table**: parses the module output and creates a list of lists with values aka 2D array aka table
